In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # INFO, WARNING messages are not printed
import tensorflow as tf
import time # for throughput measurements

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Number of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

# uninstall sionna first
import sys
from pathlib import Path

sys.path.append(str(Path('..')))

from sionna.fec.ldpc import *
from sionna.utils import BinarySource 
from sionna.utils.metrics import count_block_errors
from sionna.channel import Pauli, BinarySymmetricChannel
from sionna.utils.plotting import PlotBER

from tensorflow.keras.losses import BinaryCrossentropy
from sionna.utils.metrics import compute_bler
from sionna.fec.utils import int_mod_2, row_echelon

import time

Number of GPUs available : 1
Only GPU number 0 used.


In [2]:
GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24

num_iter = tf.constant(100)
factor=tf.constant(0.8)

bp4_decoder = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=factor, cn_type="minsum", trainable=False, stage_one=True)
osd0_decoder = OSD0_Decoder(code.N)

bp4_osd0_model = BP4_OSD_Model(code, bp4_decoder, osd0_decoder)

bp2_decoder = LDPCBPDecoder(code.hx, is_syndrome=True, hard_out=False, cn_type="minsum", num_iter=100, normalization_factor=factor)
bp2_osd0_model = BP2_OSD_Model(code.hx, code.hx_basis, code.pivot_hx, code.lx, bp2_decoder, osd0_decoder)

In [3]:
p_range = np.arange(0.08,0.101,0.01)[::-1]

ber_plot = PlotBER("Performance of the [[882,24]] code on Depolarizing channel under BP4+OSD0 Decoding")

ber_plot.simulate(bp4_osd0_model, 
                  ebno_dbs=p_range, # physical error rates to simulate
                  legend=f"{code.name}, factor={factor}, iter={num_iter}", # legend string for plotting
                  max_mc_iter=1000, # run 1000 Monte Carlo runs per physical error rate point
                  num_target_block_errors=100, # continue with next physical error rate point after 1000 block errors
                  batch_size=50000, # batch-size per Monte Carlo run
                  soft_estimates=False, # the model returns hard-estimates
                  early_stop=True, # stop simulation if no error has been detected at current physical error rate
                  show_fig=False, # do not show the figure after all results are simulated
                  add_bler=True, # we are interested in block error rate
                  qldpc=False, # since there is no flagged error for bp-osd
                  forward_keyboard_interrupt=True, # should be True in a loop
                  graph_mode=None);

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
      0.1 | 7.5139e-05 | 3.7000e-04 |        1082 |    14400000 |          111 |      300000 |        88.2 |reached target block errors
     0.09 | 1.1973e-05 | 6.0000e-05 |         977 |    81600000 |          102 |     1700000 |       350.0 |reached target block errors
     0.08 | 1.7424e-06 | 8.6580e-06 |         966 |   554400000 |          100 |    11550000 |      1845.9 |reached target block errors


In [3]:
p_range = np.arange(0.04,0.051,0.01)[::-1]

ber_plot = PlotBER("Performance of the [[882,24]] code on BSC channel under BP2+OSD0 Decoding")

ber_plot.simulate(bp2_osd0_model, 
                  ebno_dbs=p_range, # physical error rates to simulate
                  legend=f"{code.name}, factor={factor}, iter={num_iter}", # legend string for plotting
                  max_mc_iter=1000, # run 1000 Monte Carlo runs per physical error rate point
                  num_target_block_errors=100, # continue with next physical error rate point after 1000 block errors
                  batch_size=50000, # batch-size per Monte Carlo run
                  soft_estimates=False, # the model returns hard-estimates
                  early_stop=True, # stop simulation if no error has been detected at current physical error rate
                  show_fig=False, # do not show the figure after all results are simulated
                  add_bler=True, # we are interested in block error rate
                  qldpc=False, # since there is no flagged error for bp-osd
                  forward_keyboard_interrupt=True, # should be True in a loop
                  graph_mode=None);

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     0.05 | 2.1208e-04 | 5.8500e-04 |        1018 |     4800000 |          117 |      200000 |        43.6 |reached target block errors
     0.04 | 7.1802e-06 | 2.3256e-05 |         741 |   103200000 |          100 |     4300000 |       515.9 |reached target block errors


## Compared with [BP+OSD](https://github.com/quantumgizmos/bp_osd/tree/main) on Github. Their code is written in Cython, the c++ component is [here](https://github.com/quantumgizmos/ldpc). The c++ part is not using AVX, but fast due to the [mod2sparse](https://github.com/quantumgizmos/ldpc/blob/1714314b1904c1430ae79b48be8a3c7015952bd5/src/ldpc/include/mod2sparse.c).

In [4]:
from ldpc import bposd_decoder
from bposd.css import css_code

p = 0.05
bpd=bposd_decoder(
    code.hx,# the parity check matrix
    error_rate=p,
    channel_probs=[None], # assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=100, # the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0.8, # min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd0", # the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=0 # the osd search depth
)

start_time = time.perf_counter()
num_err = 0
num_samples = 100000
for i in range(num_samples):
    err = np. random.uniform(size=code.N) < p
    syndrome = code.hx @ err % 2
    bpd.decode(syndrome)
    residual_error = (bpd.osdw_decoding + err) % 2
    a = (code.lx @ residual_error % 2).any()
    num_err += a
end_time = time.perf_counter()
print(f"p={p}, elapsed time: {end_time-start_time}")
print("error rate:", num_err/num_samples)

p = 0.04
bpd=bposd_decoder(
    code.hx,# the parity check matrix
    error_rate=p,
    channel_probs=[None], # assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=100, # the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0.8, # min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd0", # the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=0 # the osd search depth
)
start_time = time.perf_counter()
num_err = 0
num_samples = 100000
for i in range(num_samples):
    err = np. random.uniform(size=code.N) < p
    syndrome = code.hx @ err % 2
    bpd.decode(syndrome)
    residual_error = (bpd.osdw_decoding + err) % 2
    a = (code.lx @ residual_error % 2).any()
    num_err += a
end_time = time.perf_counter()
print(f"p={p}, elapsed time: {end_time-start_time}")
print("error rate:", num_err/num_samples)

p=0.05, elapsed time: 60.73390205600299
error rate: 0.00065
p=0.04, elapsed time: 39.90716636599973
error rate: 0.0001


In [19]:
start_time = time.perf_counter()

GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24
# GHP_n1270_k28 = create_QC_GHP_codes(127, np.array([[0,-1,51,52,-1],[-1,0,-1,111,20],[0,-1,98,-1,122],[0,80,-1,119,-1],[-1,0,5,-1,106]]), [0,1,7], name="GHP_n1270_k28") # 16 <= d <= 46
# code = GHP_n1270_k28
num_iter = tf.constant(120)
factor=tf.constant(0.8)

decoder_hard = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=factor, cn_type="minsum", trainable=False, stage_one=False)
decoder_soft = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=factor, cn_type="minsum", trainable=False, stage_one=True)

bp4_model = BP4_Error_Model(code, decoder_hard, num_iter=num_iter, trainable=False, wt=False)
osd_model = OSD_Model(code, decoder_soft, num_iter=num_iter, trainable=False, wt=False)
bp4_osd_model = BP4_OSD_Model(code, bp4_model, osd_model)
batch_size = tf.constant(50000)
p = tf.constant(0.09)

noise_x, noise_z, x_hat, z_hat, err = bp4_model(batch_size, p)
nx, nz = noise_x[err], noise_z[err]
new_bs = tf.reduce_sum(tf.cast(err, tf.int32))
# x_hat_osd, z_hat_osd = osd_model(nx, nz, ebno_db, new_bs)
print(new_bs)
for i in range(3):
    start_time = time.perf_counter()
    osd_model(nx, nz, p, new_bs)
    end_time = time.perf_counter()
    print("osd Elapsed time: ", end_time-start_time)

for i in range(3):
    start_time = time.perf_counter()
    bp4_model(batch_size, p)
    end_time = time.perf_counter()
    print("bp Elapsed time: ", end_time-start_time)
    
for i in range(3):
    start_time = time.perf_counter()
    bp4_osd_model(batch_size, p)
    end_time = time.perf_counter()
    print("bp+osd Elapsed time: ", end_time-start_time)


tf.Tensor(649, shape=(), dtype=int32)
osd Elapsed time:  5.737122964113951
osd Elapsed time:  3.777888922020793
osd Elapsed time:  3.778188056079671
bp Elapsed time:  3.194158664904535
bp Elapsed time:  3.9569633579812944
bp Elapsed time:  3.9650151561945677
bp+osd Elapsed time:  11.926026392029598
bp+osd Elapsed time:  9.959186625899747
bp+osd Elapsed time:  9.73245100188069


## Quaternary BP
| CN update       | $p$  | $p_L$  |
| --------------- | :-:  | :-:    |
| SP, 1.0, 64     | 0.10 | 5.1e-3
|                 | 0.09 | 1.74e-3
|                 | 0.08 | 1.08e-3
| NMS, 0.8, 100   | 0.10 | 2.8e-4
|                 | 0.09 | 2e-5 (1/50000)


## Binary BP
| CN update       | $p$  | $p_L$  |
| --------------- | :-:  | :-:    |
| NMS, 0.8, 100   | 0.05 | 6.4e-4
| ^^              | 0.04 | 2e-5 (1/50000)